In [1]:
from numpy import *
import matplotlib.pyplot as plt
import operator
import time


In [12]:
def selectJ(i, oS, Ei):
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1, Ei]
    validEcacheList = nonzero(oS.eCache[:, 0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:
            if k == i:
                continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def calcEk(oS, k):
    fXk = float(multiply(oS.alphas, oS.labelMat).T * (oS.X * oS.X[k, :].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

def selectJrand(i, m):
    j = i
    while (j == i):
        j = int(random.uniform(0, m))
    return j

In [19]:
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    with open(fileName) as fr:
        for line in fr.readlines():
            lineArr = line.strip().split('    ')
            dataMat.append([float(lineArr[0]), float(lineArr[1])])
            labelMat.append(float(lineArr[2]))
    return dataMat, labelMat


class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m, 1)))
        self.b = 0
        self.eCache = mat(zeros((self.m, 2)))

trainDataSet, trainLabel = loadDataSet('testSet.txt')
dataMatIn = trainDataSet
classLabels = trainLabel
C = 0.6
toler = 0.0001 
maxIter = 40
kTup=('lin', 0)


oS = optStruct(mat(dataMatIn), mat(classLabels).transpose(), C, toler)
iterr = 0
entireSet = True
alphaPairsChanged = 0

In [17]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        j, Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if (L == H):
            # print("L == H")
            return 0
        eta = 2.0 * oS.X[i, :] * oS.X[j, :].T - oS.X[i, :] * oS.X[i, :].T - oS.X[j, :] * oS.X[j, :].T
        if eta >= 0:
            # print("eta >= 0")
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, j)
        if (abs(oS.alphas[j] - alphaJold) < 0.00001):
            # print("j not moving enough")
            return 0
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS, i)
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[i, :].T - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[i, :] * oS.X[j, :].T
        b2 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[j, :].T - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[j, :] * oS.X[j, :].T
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

In [18]:
i = 0
innerL(i, oS)

0

In [20]:
Ei = calcEk(oS, i)
Ei

1.0

In [21]:
oS.labelMat

matrix([[-1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [-

In [22]:
oS.alphas

matrix([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      

In [25]:
k = i

In [40]:
b = 0; m,n = shape(dataMatrix)
alphas = mat(zeros((m,1)))

In [23]:
multiply(oS.alphas, oS.labelMat).T

matrix([[-0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,
          0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
          0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,
          0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
         -0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,
         -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,
          0.,  0., -0., -0., -0., -0., -0., -0., -0.]])

In [36]:
dataMatrix = mat(dataMatIn)

In [37]:
(dataMatrix*dataMatrix[i,:].T)

matrix([[ 16.45930283],
        [ 15.74944568],
        [ 23.62676274],
        [  7.48352117],
        [ 31.31011654],
        [ 23.23121223],
        [ 34.5502158 ],
        [  8.76530388],
        [ 16.29538278],
        [ -0.56047978],
        [ 11.65350847],
        [  5.97217245],
        [ -2.38720614],
        [ 20.68613682],
        [  3.61193398],
        [ 29.98735792],
        [ 29.57379773],
        [ 23.43708958],
        [ 32.09373166],
        [  4.70087189],
        [  6.19516763],
        [  7.67904154],
        [ 28.85126395],
        [ 10.32526493],
        [ 17.18557231],
        [ 26.00106749],
        [ 26.30828953],
        [  7.09568772],
        [ -1.90677766],
        [ 12.08413697],
        [ 10.59876818],
        [ 30.14366779],
        [ 33.67857529],
        [ 24.52205584],
        [ 26.55162967],
        [ 10.1555109 ],
        [ 20.79607383],
        [  4.29629731],
        [  6.03878151],
        [ 37.20946573],
        [  3.23741434],
        [ 10.139

In [ ]:
multiply(oS.alphas, oS.labelMat).T *

In [41]:
i

0

In [43]:
labelMat = oS.labelMat

In [44]:
multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)

matrix([[0.]])

In [33]:
oS.X[k, :]

matrix([[3.542485, 1.977398]])

In [34]:
oS.X

matrix([[ 3.542485e+00,  1.977398e+00],
        [ 3.018896e+00,  2.556416e+00],
        [ 7.551510e+00, -1.580030e+00],
        [ 2.114999e+00, -4.466000e-03],
        [ 8.127113e+00,  1.274372e+00],
        [ 7.108772e+00, -9.869060e-01],
        [ 8.610639e+00,  2.046708e+00],
        [ 2.326297e+00,  2.652130e-01],
        [ 3.634009e+00,  1.730537e+00],
        [ 3.413670e-01, -8.949980e-01],
        [ 3.125951e+00,  2.932510e-01],
        [ 2.123252e+00, -7.835630e-01],
        [ 8.878350e-01, -2.797792e+00],
        [ 7.139979e+00, -2.329896e+00],
        [ 1.696414e+00, -1.212496e+00],
        [ 8.117032e+00,  6.234930e-01],
        [ 8.497162e+00, -2.666490e-01],
        [ 4.658191e+00,  3.507396e+00],
        [ 8.197181e+00,  1.545132e+00],
        [ 1.208047e+00,  2.131000e-01],
        [ 1.928486e+00, -3.218700e-01],
        [ 2.175808e+00, -1.452700e-02],
        [ 7.886608e+00,  4.617550e-01],
        [ 3.223038e+00, -5.523920e-01],
        [ 3.628502e+00,  2.190585e+00],


In [27]:
oS.b

0

In [30]:
fXk

0.0

In [26]:
(oS.X * oS.X[k, :].T)

matrix([[ 16.45930283],
        [ 15.74944568],
        [ 23.62676274],
        [  7.48352117],
        [ 31.31011654],
        [ 23.23121223],
        [ 34.5502158 ],
        [  8.76530388],
        [ 16.29538278],
        [ -0.56047978],
        [ 11.65350847],
        [  5.97217245],
        [ -2.38720614],
        [ 20.68613682],
        [  3.61193398],
        [ 29.98735792],
        [ 29.57379773],
        [ 23.43708958],
        [ 32.09373166],
        [  4.70087189],
        [  6.19516763],
        [  7.67904154],
        [ 28.85126395],
        [ 10.32526493],
        [ 17.18557231],
        [ 26.00106749],
        [ 26.30828953],
        [  7.09568772],
        [ -1.90677766],
        [ 12.08413697],
        [ 10.59876818],
        [ 30.14366779],
        [ 33.67857529],
        [ 24.52205584],
        [ 26.55162967],
        [ 10.1555109 ],
        [ 20.79607383],
        [  4.29629731],
        [  6.03878151],
        [ 37.20946573],
        [  3.23741434],
        [ 10.139

In [31]:
oS.labelMat[k]

matrix([[-1.]])

In [29]:
fXk = float(multiply(oS.alphas, oS.labelMat).T * (oS.X * oS.X[k, :].T)) + oS.b
Ek = fXk - float(oS.labelMat[k])

In [32]:
Ek

1.0

In [ ]:

if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
    j, Ej = selectJ(i, oS, Ei)
    alphaIold = oS.alphas[i].copy()
    alphaJold = oS.alphas[j].copy()
    if (oS.labelMat[i] != oS.labelMat[j]):
        L = max(0, oS.alphas[j] - oS.alphas[i])
        H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
    else:
        L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
        H = min(oS.C, oS.alphas[j] + oS.alphas[i])
    if (L == H):
        # print("L == H")
        return 0
    eta = 2.0 * oS.X[i, :] * oS.X[j, :].T - oS.X[i, :] * oS.X[i, :].T - oS.X[j, :] * oS.X[j, :].T
    if eta >= 0:
        # print("eta >= 0")
        return 0
    oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
    oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
    updateEk(oS, j)
    if (abs(oS.alphas[j] - alphaJold) < 0.00001):
        # print("j not moving enough")
        return 0
    oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
    updateEk(oS, i)
    b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[i, :].T - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[i, :] * oS.X[j, :].T
    b2 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[j, :].T - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[j, :] * oS.X[j, :].T
    if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
        oS.b = b1
    elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
        oS.b = b2
    else:
        oS.b = (b1 + b2) / 2.0
    return 1
else:
    return 0